# Test of using all parameters

In [27]:
# Specify some paths and names in the cell below.
config_path = "T:/Data/"  # Path to folder where you saved your wildlifeconfig file.
filter_set_json = None
taxon_json = None
query_name = 'All_params_1'
ask_eBird = True
ask_GBIF = False
get_dwca = False 

In [28]:
# Nothing to fill out in this cell
%matplotlib inline
import os
import sqlite3
import pprint
import json
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import sys
sys.path.append(config_path)
import wranglerconfig as config
sys.path.append(config.codeDir)
import wrangler_functions as functions

# Define some variables
t1 = datetime.now()
working_directory = config.workDir
username = config.gbif_username
password = config.gbif_password
email = config.gbif_email
EBD_file = config.EBD_file
output_database = working_directory + query_name + '.sqlite'

# Set display options
pd.set_option('display.width', 600)
pd.set_option('display.max_colwidth', 75)
pd.set_option('display.max_rows', 150)

print("Notebook run " + str(t1))
print("Results were saved in " + output_database)

Notebook run 2021-05-28 14:31:12.428443
Results were saved in T:/Occurrence_Records/All_params_1.sqlite


## Taxon Concept
Caution! -- Taxon concept mismatches can cause inclusion of innapropriate records or ommission of useful records.

In [29]:
your_taxon_id = "bwitux0"
gbif_id = 9606290 
ebird_id = "Wild Turkey"
detection_distance_m = 300
taxon_polygon = None

In [30]:
# If a json was provided, use it, otherwise create a new one with info that was provided.
if taxon_json is None:
    # Build a species dictionary
    taxon_info = {"ID": your_taxon_id, "GBIF_ID": gbif_id, "EBIRD_ID": ebird_id, "detection_distance_m": detection_distance_m,
                  "TAXON_EOO": taxon_polygon}

    # Save as json object
    out_file = open(working_directory + your_taxon_id + ".json", "w")  
    json.dump(taxon_info, out_file) 
    out_file.close() 

if taxon_json is not None:
    with open(taxon_json, "r") as f:
        taxon_info = json.load(f)
        f.close()
        
pprint.pprint(taxon_info)

{'EBIRD_ID': 'Wild Turkey',
 'GBIF_ID': 9606290,
 'ID': 'bwitux0',
 'TAXON_EOO': None,
 'detection_distance_m': 300}


## Filter Set
Specify how you want records filtered and cleaned.  Alternatively, you can load a filter set here by specifying a path in the first cell of this notebook.  

To skip a filter, enter "None" without the quotation marks or "".  

In [31]:
filter_set_name = "Test_EBD_1"
print("Filter set name: " + str(filter_set_name))

Filter set name: Test_EBD_1


#### GBIF Request Method

In [32]:
"""
Whether to get records from GBIF in a darwin core archive.  
"False" uses the GBIF API, which has limitations that may be important.  
"True" requests results be emailed in a darwin core archive.
"""
print("Request a Darwin Core Archive? " + str(get_dwca))

Request a Darwin Core Archive? False


#### Date Limits
Notes: 

In [33]:
'''
Enter year and month ranges.  For example, years_range = 2015,2017 and months_range = 3,6
'''
years_range = "2018,2021"
months_range = "5,12"
print("Years: " + str(years_range))
print("Months: " + str(months_range))

Years: 2018,2021
Months: 5,12


#### Country
Notes:

In [34]:
country = "US"
print("Country: " + country)

Country: US


#### Bounding Box
Notes:

In [35]:
'''
Coordinates should correspond to WGS84 (EPSG:4326).  Don't use this option if you specify a query polygon below.
'''
lat_range = "20,50"
lon_range = "-130,-67"
print("Latitude range: " + lat_range)
print("Longitude range: " + lon_range)

Latitude range: 20,50
Longitude range: -130,-67


#### Area of Interest
Notes:

In [36]:
'''
Specify polygons to use for spatial filtering.  
Records with coordinates outside of the polygons will be removed.  
You can specify a geometry for the query and one for the species.  
The species geometry is included to facilitate better handling of taxonomic issues.  
If both are provided, the intersection is calculated and used as the filter.  
The format should be well known text in WGS84 (EPSG 4326), and very importantly, vertices need to be listed counter-clockwise.  
See the ccw_wkt_from_shp() function in wrangler functions for help.
'''
query_polygon = None
print(query_polygon)

None


#### Taxon EOO
Notes:

In [37]:
'''
True or False whether you want to apply the taxon EOO to the filtering.  
If True, removes records with centroids outside of the extent of occurrence geometry you provided in taxon_info.
'''
use_taxon_geometry = False
print("Use taxon extent of occurrence? " + str(use_taxon_geometry))

Use taxon extent of occurrence? False


#### Geoissue
Notes:

In [38]:
'''
Are GBIF records with noted geoissues OK to include? GBIF only.
'''
geoissue = None
print("Records with geoissues OK? " + str(geoissue))

Records with geoissues OK? None


#### Collections
Notes:

In [39]:
'''
List collection codes that you'd like to omit. GBIF only.
'''
collection_codes_omit = ["naturgucker", "VBBA2"]
print("Omit: " + str(collection_codes_omit))

Omit: ['naturgucker', 'VBBA2']


#### Institutions
Notes:

In [40]:
'''
List institution codes that you'd like to omit. GBIF only
'''
institutions_omit = ["e3aa237e-53c8-4c46-93ef-77a18bb9c80e", "urn:lsid:biocol.org:col:34939"]
print("Omit: " + str(institutions_omit))

Omit: ['e3aa237e-53c8-4c46-93ef-77a18bb9c80e', 'urn:lsid:biocol.org:col:34939']


#### Datasets
Notes:

In [41]:
'''
List datasets that you'd like to omit.
'''
datasets_omit = ["Gaia Guide", "Earth Guardians Weekly Feed"]
print("Omit: " + str(datasets_omit))

Omit: ['Gaia Guide', 'Earth Guardians Weekly Feed']


#### Coordinate Uncertainty
Notes:

In [42]:
'''
Do you want to remove records without coordinate uncertainty (True) or leave them in the data set (False)?  
Note that eBird records in GBIF (EOD) do not have this and neither do data in the EBD dataset.  
With the EBD, the length of traveling counts is used as a surrogate value.  

max_coordinate_uncertainty must be an integer greater than 0.

default_coordUncertainty -- coordinateUncertaintyInMeters is often not provided.  
Here is an option to use a default.  If you don't want anything entered, set this equal to False (boolean, not string).

A maximum for coordinate uncertainty can also be set in meters.
'''
has_coordinate_uncertainty = True
default_coordUncertainty = 2000
max_coordinate_uncertainty = 10000
print("Coordinate uncertainty required? " + str(has_coordinate_uncertainty))
print("Default coordinate uncertainty to use: " + str(default_coordUncertainty))
print("Maximum allowable coordinate uncertainty: " + str(max_coordinate_uncertainty))

Coordinate uncertainty required? True
Default coordinate uncertainty to use: 2000
Maximum allowable coordinate uncertainty: 10000


#### Bases
Notes:

In [43]:
'''
List bases of records that you want to omit.  GBIF only.
'''
bases_omit = ["PRESERVED_SPECIMEN", "MACHINE_OBSERVATION"]
print("Omit: " + str(bases_omit))

Omit: ['PRESERVED_SPECIMEN', 'MACHINE_OBSERVATION']


#### Sampling Protocols
Notes:

In [44]:
'''
List sampling protocols that you would like to omit.
'''
sampling_protocols_omit = ["eBird Pelagic Protocol", "Random"]
print("Omit: " + str(sampling_protocols_omit))

Omit: ['eBird Pelagic Protocol', 'Random']


#### Issues
Notes:

In [45]:
'''
List issues that you want to omit.  GBIF only.
'''
issues_omit = ["TAXON_MATCH_HIGHERRANK", "COORDINATE_UNCERTAINTY_METERS_INVALID"]
print("Omit: " + str(issues_omit))

Omit: ['TAXON_MATCH_HIGHERRANK', 'COORDINATE_UNCERTAINTY_METERS_INVALID']


#### Duplicates
Notes:

In [46]:
'''
Specify whether duplicates on latitude, longitude, and date should be included.
'''
duplicates_OK = False
print("Allow duplicates? " + str(duplicates_OK))

Allow duplicates? False


#### Filter Set Summary

In [48]:
if filter_set_json is None:
    # Build a filter set dictionary
    filter_set = {"name": filter_set_name, "query_polygon": query_polygon, "issues_omit": issues_omit,
                  "sampling_protocols_omit": sampling_protocols_omit, "bases_omit": bases_omit,
                  "has_coordinate_uncertainty": has_coordinate_uncertainty, "geoissue": geoissue,
                  "default_coordUncertainty": default_coordUncertainty,
                  "max_coordinate_uncertainty": max_coordinate_uncertainty,
                  "datasets_omit": datasets_omit, "collection_codes_omit": collection_codes_omit,
                  "institutions_omit": institutions_omit, "geoissue": geoissue, "use_taxon_geometry": use_taxon_geometry,
                  "lat_range": lat_range, "lon_range": lon_range, "country": country, 
                  "years_range": years_range, "months_range": months_range, "duplicates_OK": duplicates_OK, "get_dwca": get_dwca}
    
    # Replace empty strings with None
    for x in filter_set.keys():
        if filter_set[x] == "":
            filter_set[x] = None
    
    # Save as json object
    with open(working_directory + filter_set_name + ".json", "w") as f:
        json.dump(filter_set, f) 
        f.close()
        
if filter_set_json is not None:
    with open(filter_set_json, "r") as f:
        filter_set = json.load(f)
        f.close()
        
    # Replace empty strings with None
    for x in filter_set.keys():
        if filter_set[x] == "":
            filter_set[x] = None
        
pprint.pprint(filter_set)

{'bases_omit': ['PRESERVED_SPECIMEN', 'MACHINE_OBSERVATION'],
 'collection_codes_omit': ['naturgucker', 'VBBA2'],
 'country': 'US',
 'datasets_omit': ['Gaia Guide', 'Earth Guardians Weekly Feed'],
 'default_coordUncertainty': 2000,
 'duplicates_OK': False,
 'geoissue': None,
 'get_dwca': False,
 'has_coordinate_uncertainty': True,
 'institutions_omit': ['e3aa237e-53c8-4c46-93ef-77a18bb9c80e',
                       'urn:lsid:biocol.org:col:34939'],
 'issues_omit': ['TAXON_MATCH_HIGHERRANK',
                 'COORDINATE_UNCERTAINTY_METERS_INVALID'],
 'lat_range': '20,50',
 'lon_range': '-130,-67',
 'max_coordinate_uncertainty': 10000,
 'months_range': '5,12',
 'name': 'Test_EBD_1',
 'query_polygon': None,
 'sampling_protocols_omit': ['eBird Pelagic Protocol', 'Random'],
 'use_taxon_geometry': False,
 'years_range': '2018,2021'}


## Processing

In [50]:
# Create an output database
functions.build_output_database(output_database)

# Save taxon and filter set info into database
output_db_conn= sqlite3.connect(output_database)
cursor = output_db_conn.cursor()
pd.DataFrame(taxon_info.values(), taxon_info.keys()).applymap(str).to_sql(name='taxon_concept', con=output_db_conn, if_exists='replace')
pd.DataFrame(filter_set.values(), filter_set.keys()).applymap(str).to_sql(name='filter_set', con=output_db_conn, if_exists='replace')
output_db_conn.close()

In [51]:
# Run the appropriate queries
if ask_eBird == True and ask_GBIF == True:
    # Run eBird query
    ebird_data = functions.get_EBD_records(taxon_info, filter_set, working_directory, EBD_file, query_name)
    # Run GBIF query
    gbif_data = functions.get_GBIF_records(taxon_info, filter_set, query_name, working_directory, username, password, email) 

elif ask_eBird == True and ask_GBIF == False:
    # Run eBird query
    ebird_data = functions.get_EBD_records(taxon_info, filter_set, working_directory, EBD_file, query_name)
    gbif_data = None

elif ask_eBird == False and ask_GBIF == True:
    # Run GBIF query
    gbif_data = functions.get_GBIF_records(taxon_info, filter_set, query_name, working_directory, username, password, email)
    ebird_data = None

Ran EBD query with Auk: 0:27:23.619058
Calculated the spatial filter polygon: 0:00:02.007671
Applied spatial filter: 0:00:00
Summarized fields returned: 0:00:00.186977
Prepared the eBird records for processing: 0:00:06.251169


In [52]:
# Filter out records with undesirable values, locations, and/or duplication.
#import importlib
#importlib.reload(functions)
for x in filter_set.keys():
    if filter_set[x] == "":
        filter_set[x] = None

In [53]:
functions.process_records(ebird_data=ebird_data, gbif_data=gbif_data, filter_set=filter_set, 
                        taxon_info=taxon_info, working_directory=working_directory, 
                        query_name=query_name)

Prepared data frames for processing: 0:00:03.147511
Summarized values acquired: 0:00:00.623346
Number of georeferenced records: 163610
Number of record without georeference: 62068
Applying default coordinate uncertainties
AN ERROR OCCURRED !!!!!!!!!!!!!
Performed filtering: 0:00:01.503516


ValueError: invalid literal for int() with base 10: 'X'

In [ ]:
output_db_conn= sqlite3.connect(output_database)
cursor = output_db_conn.cursor()
record_count = cursor.execute("SELECT COUNT(record_id) FROM occurrence_records;").fetchone()
print(str(record_count[0]) + " records were saved in the output database")

## Results of the Filtering

#### Attributes Returned for GBIF Records
This count was made before filters were applied

In [ ]:
if ask_GBIF == True:
    fields_summary = pd.read_sql("SELECT * FROM gbif_fields_returned", output_db_conn)
    fields_summary.index.name = 'Field'
    pd.set_option('display.max_rows', 250)
    print(fields_summary.sort_values(by="attribute"))
if ask_GBIF == False:
    print("GBIF was not queried.")

#### Attributes Returned for eBird Records
This count was made before filters were applied

In [ ]:
if ask_eBird == True:
    fields_summary = pd.read_sql("SELECT * FROM ebird_fields_returned", output_db_conn)
    fields_summary.index.name = 'Field'
    pd.set_option('display.max_rows', 250)
    print(fields_summary)
if ask_eBird == False:
    print("No eBird Basic Dataset was queried.")

#### Sources

In [ ]:
sources = pd.read_sql(sql="SELECT * FROM sources;", con=output_db_conn)
print(sources[['institutionID', 'collectionCode', 'datasetName', 'acquired', 'removed', 'retained']])

#### Bases

In [ ]:
bases = pd.read_sql(sql="SELECT * FROM attribute_value_counts WHERE attribute = 'basisOfRecord';", con=output_db_conn)
print(bases[['value', 'acquired', 'removed', 'retained']])

#### Protocols

In [ ]:
protocols = pd.read_sql(sql="SELECT * FROM attribute_value_counts WHERE attribute = 'samplingProtocol';", con=output_db_conn)
if protocols.empty == True:
    print("No protocols were documented.")
if protocols.empty == False:
    print(protocols[['value', 'acquired', 'removed', 'retained']])

#### Issues

In [ ]:
issues = pd.read_sql(sql="SELECT * FROM attribute_value_counts WHERE attribute = 'issues';", con=output_db_conn)
if issues.empty == True:
    print("No issues were documented.")
if issues.empty == False:
    print(issues[['value', 'acquired', 'removed', 'retained']])

#### Establishment Means

In [ ]:
establishment = pd.read_sql(sql="SELECT * FROM attribute_value_counts WHERE attribute = 'establishmentMeans';", con=output_db_conn)
if establishment.empty == True:
    print("No establishment means were reported.")
if establishment.empty == False:
    print(establishment[['value', 'acquired', 'removed', 'retained']])

#### Identification Qualifiers

In [ ]:
qualifications = pd.read_sql(sql="SELECT * FROM attribute_value_counts WHERE attribute = 'identificationQualifers';", con=output_db_conn)
if qualifications.empty == True:
    print("No identification qualifiers were reported.")
if qualifications.empty == False:
    print(establishment[['value', 'acquired', 'removed', 'retained']])

## Descriptions of Retained Records

#### Locations

In [ ]:
# Get the record coordinates as a data frame
record_coordinates = (pd.read_sql("""SELECT decimalLatitude, decimalLongitude, radius_meters
                                     FROM occurrence_records""",
                                  con=output_db_conn)
                      .astype({'decimalLongitude': 'float', 'decimalLatitude': 'float',
                               'radius_meters': 'float'}))

# Make the data frame spatial
gdf = gpd.GeoDataFrame(record_coordinates, geometry=gpd.points_from_xy(record_coordinates['decimalLongitude'],
                                                   record_coordinates['decimalLatitude']))

# Set the coordinate reference system
gdf.crs={'init' :'epsg:4326'}

# Create world map <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
base = world.plot(figsize=(12,12), color = 'darkkhaki')
gdf.plot(ax=base, marker='o', color='k', markersize=5)
plt.show()

# Create USA map <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
if filter_set["country"] == "US":
    usa_bbox = np.array([-124.725839,   24.498131,  -66.949895,   49.384358])
    fig, ax = plt.subplots(figsize=(12,12))
    ax.set_xlim(([usa_bbox[0],  usa_bbox[2]]))
    ax.set_ylim(([usa_bbox[1],  usa_bbox[3]]))
    world.plot(ax=ax, color='darkkhaki')
    gdf.plot(ax=ax, marker='o', color='k', markersize=5)
    plt.show()

# Create coordinate extent map <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
os.chdir(config.codeDir)
states = gpd.read_file(os.getcwd() + '/data/us_states.shp')

# Reproject states and record coordinates to facilitate buffering
states = states.to_crs(epsg=5070)
footprints = gdf.to_crs(epsg=5070)

# Buffer points for record footprints
footprints['footprint']=footprints.apply(lambda x: x.geometry.buffer(x.radius_meters), axis=1)
footprints.set_geometry(col='footprint', inplace=True, drop=True)

# Map the buffered points/footprints
fig, ax = plt.subplots(figsize=(12,12))
coordinate_bbox = footprints.geometry.total_bounds
ax.set_xlim(([coordinate_bbox[0],  coordinate_bbox[2]]))
ax.set_ylim(([coordinate_bbox[1],  coordinate_bbox[3]]))
states.plot(ax=ax, color = 'darkkhaki')
footprints.boundary.plot(ax=ax, color='k')
plt.show()

# Cleanup
del footprints, gdf

#### Years Represented

In [ ]:
occ_years = [int(x[0]) for x in cursor.execute("SELECT strftime('%Y', eventDate) FROM occurrence_records").fetchall()]
years = filter_set['years_range']
years = years.split(',')
yearsrng = list(range(int(years[0]), int(years[1]), 1))
binsnum = int(years[1]) - int(years[0])
plt.hist(occ_years, bins=binsnum)
plt.ylabel("number of records")
plt.xlabel("year")
plt.xticks(yearsrng, rotation=90)
plt.title("Occurrences per Year")

#### Months Represented

In [ ]:
occ_months = [int(x[0]) for x in cursor.execute("SELECT strftime('%m', eventDate) FROM occurrence_records").fetchall()]
plt.hist(occ_months, bins=range(1, 14), color="g")
plt.ylabel("number of records")
plt.xlabel("month")
plt.xticks(range(1, 13))
plt.title("Occurrences per Month")

#### Distribution of Coordinate Uncertainty Values for Retained Records

In [ ]:
occ_cert = [float(x[0]) for x in cursor.execute("SELECT coordinateUncertaintyInMeters FROM occurrence_records").fetchall()]
maxi = max(occ_cert)
plt.figure(figsize=(16,4))
plt.hist(occ_cert, bins=50, color="r")
plt.xticks(range(0, int(maxi), int(maxi/50)), rotation=90)
plt.ylabel("number of records")
plt.xlabel("meters")
plt.title("Coordinate Uncertainty")

In [ ]:
occ_cert = [float(x[0]) for x in cursor.execute("SELECT coordinateUncertaintyInMeters FROM occurrence_records").fetchall()]
plt.figure(figsize=(11,4))
plt.boxplot(occ_cert, vert=False)
plt.xlabel("meters")
plt.title("Coordinate Uncertainty")

In [ ]:
rng_max = 2000
occ_cert2 = [x for x in occ_cert if x <= rng_max]
plt.figure(figsize=(12,4))
plt.hist(occ_cert2, bins=30, color="m", align='mid')
plt.xticks(range(0, rng_max + 100, int(rng_max/30.)), rotation=90)
plt.ylabel("number of records")
plt.xlabel("meters")
plt.title("Coordinate Uncertainties Below 2km")

#### Remarks

In [ ]:
print("General remarks:")
remarks = output_db_conn.execute("SELECT DISTINCT general_remarks FROM occurrence_records;").fetchall()
if len(remarks) <= 20:
    try:
        for rem in remarks:
            if rem[0][0:1] == ';':
                print(rem[0][2:])
            else:
                print(rem[0])
    except:
        pass
else:
    print("More than 20 remarks, consult the occurrence database.")

In [ ]:
print("Event remarks:")
remarks = output_db_conn.execute("SELECT DISTINCT eventRemarks FROM occurrence_records;").fetchall()
if len(remarks) <= 20:
    try:
        for rem in remarks:
            if rem[0][0:1] == ';':
                print(rem[0][2:])
            else:
                print(rem[0])
    except:
        pass
else:
    print("More than 20 remarks, consult the occurrence database.")

In [ ]:
print("Occurrence remarks:")
remarks = output_db_conn.execute("SELECT DISTINCT occurrenceRemarks FROM occurrence_records;").fetchall()
if len(remarks) <= 20:
    try:
        for rem in remarks:
            if rem[0][0:1] == ';':
                print(rem[0][2:])
            else:
                print(rem[0])
    except:
        pass
else:
    print("More than 20 remarks, consult the occurrence database.")

In [ ]:
print("Location remarks:")
remarks = output_db_conn.execute("SELECT DISTINCT locationRemarks FROM occurrence_records;").fetchall()
if len(remarks) <= 20:
    try:
        for rem in remarks:
            if rem[0][0:1] == ';':
                print(rem[0][2:])
            else:
                print(rem[0])
    except:
        pass
else:
    print("More than 20 remarks, consult the occurrence database.")

In [ ]:
print("Identified remarks:")
remarks = output_db_conn.execute("SELECT DISTINCT general_remarks FROM occurrence_records;").fetchall()
if len(remarks) <= 20:
    try:
        for rem in remarks:
            if rem[0][0:1] == ';':
                print(rem[0][2:])
            else:
                print(rem[0])
    except:
        pass
else:
    print("More than 20 remarks, consult the occurrence database.")

In [ ]:
print("Georeference remarks:")
remarks = output_db_conn.execute("SELECT DISTINCT georeferenceRemarks FROM occurrence_records;").fetchall()
if len(remarks) <= 20:
    try:
        for rem in remarks:
            if rem[0][0:1] == ';':
                print(rem[0][2:])
            else:
                print(rem[0])
    except:
        pass
else:
    print("More than 20 remarks, consult the occurrence database.")

## Citations
#### eBird 

In [ ]:
if ask_eBird == True:
    with open(config.EBD_file[:-22] + "recommended_citation.txt", "r") as reference:
        print(reference.readlines()[0])
else:
    print("No eBird Basic Dataset was queried")

#### GBIF

In [ ]:
if ask_GBIF == True:
    if get_dwca == True:
        print("Citations-- ")
        print(cursor.execute("SELECT citations FROM GBIF_download_info").fetchall()[0][0])
    else:
        print("Set 'get_dwca' to True to acquire a list of citations")
if ask_GBIF == False:
    print("GBIF was not queried.")

In [ ]:
if ask_GBIF == True:
    if get_dwca == True:
        print("Rights-- ")
        print(output_db_conn.execute("SELECT rights FROM GBIF_download_info").fetchall()[0][0])
    else:
        print("Set 'get_dwca' to True to see the rights")
if ask_GBIF == False:
    print("GBIF was not queried.")

In [ ]:
if ask_GBIF == True:
    if get_dwca == True:
        print("DOI-- ")
        doi = output_db_conn.execute("SELECT doi FROM GBIF_download_info").fetchall()[0][0]
        print("https://doi.org/" + doi)
    else:
        print("Set 'get_dwca' to True to perform a search with a doi assigned")
if ask_GBIF == False:
    print("GBIF was not queried.")

In [ ]:
if ask_GBIF == True:
    if get_dwca == True:
        print("GBIF download key-- ")
        print(output_db_conn.execute("SELECT download_key FROM GBIF_download_info").fetchall()[0][0])
    else:
        print("Set 'get_dwca' to True to perform a search with a download key assigned")
if ask_GBIF == False:
    print("GBIF was not queried")

## Runtime

In [ ]:
output_db_conn.close()
del cursor
t2 = datetime.now()
print(t2 - t1)